# Load and clean the data

In [39]:
import pandas as pd
import ezodf
import re
import os

import sys
sys.path.append("..")

import src.data_loading.loads_from_url as lfu
import src.utils.loads_from_url as utils
import src.data_cleaning.modify_dfs as mdf

cwd = os.getcwd()
cwd = re.split(r"[A-Za-z]+$",cwd)[0]

folder_path = os.path.join(cwd, 'data')

file_path = []
for file_ in os.listdir(folder_path):
    if file_.endswith('.ods') and not re.search(r'^_',file_):
        file_path.append(os.path.join(folder_path, file_) )

## Load a file

The file format changes with each file. So had to do a slightly convoluted load.

Also we want to ignore sheets with names: Introduction, Summary or including SUM.

In [ ]:
def import_ods(fname):
    """
    imports ods files given a filename and returns a pd dataframe
    used with function import_ods_inner which does the importing for each sheet != 0
    
    Args:
        fname (string): a string to location of the ods file
    
    Returns:
        a dataframe of the ods file
    
    """
    
    df = pd.DataFrame()
    
    doc = ezodf.opendoc(fname)
    
    for i, sheet in enumerate(doc.sheets):
        product = sheet.name

        if product != 'Introduction' and product != 'Summary' and not re.search(r"SUM",product): #ignore 1st sheet
            
            # main call
            df_new, bool_sheet = _import_ods_inner(sheet)
            
            # if sheet is not a bad sheet
            if bool_sheet == True:
                df_new['product'] = product
                df_new.columns = df_new.columns.str.strip()
                try:
                    if len(df_new.columns)>3:
                        df = pd.concat([df,df_new])
                    else:
                        print(f"{product} not enough columns")
                except:
                    df = df_new

                df.reset_index(inplace=True,drop=True)    
                
        else:
            fname_ = fname.split('\\')[-1].split('.')[0]
            print(f'Failed to load {fname_}: {product}')
            

    return df

def _import_ods_inner(sheet):
    """
    inner function of import_ods
    takes individual sheets and returns a pd df 
    """
    
    data_sheet = []
    got_colname =False
    for i,row in enumerate(sheet.rows()):

        if got_colname == False:
            column_names = [cell.value for cell in row]

            if column_names[0] == 'Sample ID':
                got_colname = True
                     
        else:
            data_sheet.append( [cell.value for cell in row] )
            
    
    if got_colname:
        ddf = pd.DataFrame(data_sheet)

        ddf.columns = column_names

        # delete none column
        try:
            del ddf[None]
        except:
            pass


        # fill based on previous values    
        ddf.fillna(method='ffill', inplace=True)

        return ddf,True
    else:
        return [],False

In [ ]:
df = import_ods(file_path[0])
print(f"\nLength of df is {len(df)}")
df.head()

## Load multiple files


In [ ]:
def import_all_ods(folder_path):
    """
    Imports all ods files in a folder and returns a pd df
    
    Args:
        folder_path (string): a string to location of the ods file
    
    Returns:
        pd.Dataframe (mod_df): a dataframe of all ods files combined,
            with modifcations applied
    """
    
    dict_column_names = {'Sampling Point':'Retail Outlet',
                     'Packer / Manufacturer':'Packer / Manufacturer / Importer'}
    
    
    file_path = []
    for file_ in os.listdir(folder_path):
        if file_.endswith('.ods') and not re.search(r'^_',file_):
            file_path.append(os.path.join(folder_path, file_) )
    
    all_df_lst = []
    for file_ in file_path:
        fname_ = file_.split('\\')[-1].split('.')[0]
        print(f"Importing {fname_}")
        df = import_ods(file_)
        df = df.rename(columns=dict_column_names)
        
        # put each modified df into a list
        all_df_lst.append(df)

    # concat all the modified dfs   
    df_all = pd.concat(all_df_lst)

    # modify the concatenated dfs
#     mod_df = md.modify_df(df_all)

    return df_all

all_dfs = import_all_ods(folder_path)


In [40]:
lfu.import_all_ods(folder_path)

Importing 2016_annual_data
Importing 2017_annual_data
Importing 2018_annual_data


KeyboardInterrupt: 

## Data cleaning



In [44]:
import sys
sys.path.append("..")
import os
import re
import src.data_loading.loads_from_url as lfu
cwd = os.getcwd()
cwd = re.split(r'[A-Za-z]+$',cwd)[0]
cwd = os.path.join(cwd,'data')
file = os.path.join(cwd,'combined_df.csv')
lfu.save_dfs(folder_path = cwd, file_folder=file)



Importing 2016_annual_data
Importing 2017_annual_data
Importing 2018_annual_data
Importing Apr_2022_rolling_report_v2
Importing August_2022_rolling_reports
Importing July_2022_rolling_reports
Importing June_2022_rolling_reports
Importing May_2022_rolling_report
Importing October_2022_rolling_reports
Importing Q1_2018_quarterly_data
Importing Q1_2019_quarterly_data
Importing Q1_2020_quarterly_data
Importing Q1_2021_quarterly_data
Importing Q2_2018_quarterly_data
Importing Q2_2019_quarterly_data
Importing Q2_2021_quarterly_data
Importing Q2_Q3_2020_Data
Importing Q3_2018_quarterly_data
Importing Q3_2019_quarterly_data
Importing Q3_2021_quarterly_data
Importing Q4_2018_quarterly_data
Importing Q4_2019_quarterly_data
Importing Q4_2020_Data
Importing Q4_2021_PRIF_Quarterly_Report
Importing September_2022_rolling_reports
